In [1]:
import json
import sys
import random

sys.path.append("../")

from tqdm import tqdm
from preprocessors.preprocessor import Preprocessor
from preprocessors.synonym_expander import SynonymExpander
from preprocessors.llm_expander import LLMExpander

import pandas as pd
import numpy as np

from models.bm25 import bm25
from models.vector_model import vector_model

import IR_utils

random.seed(0)
np.random.seed(0)

%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/manoschatzakis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/manoschatzakis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/manoschatzakis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
preprocessor = Preprocessor(expander=SynonymExpander(add_synonym_prob=1))

docs = preprocessor.load_docs("../../data/dataset/tokenized_corpus.jsonl")

test_queries_t1 = IR_utils.load_test_queries_t1(
    "../../data/dataset/queries.jsonl", "../../data/task1_test.tsv"
)[0]

test_queries_t2 = IR_utils.load_test_queries_t2(
    "../../data/dataset/queries.jsonl", "../../data/task2_test.tsv"
)[0]


print("Number of docs:", len(docs))
print("Number of queries (t1):", len(test_queries_t1))
print("Number of queries (t2):", len(test_queries_t2))

Number of docs: 1471406
Number of queries (t1): 7437
Number of queries (t2): 33


In [3]:
model = bm25(list(docs.keys()), list(docs.values()))
model.fit()

In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

flan_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
flan_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

llmExpander = LLMExpander(flan_model, flan_tokenizer, max_new_tokens=20)

In [20]:
csv_string = "id,corpus-id,score\n"

for query_data in tqdm(test_queries_t1, desc="Query Preprocessing and Expansion", unit=" queries"):
    query_text = query_data["text"]
    query_terms = preprocessor.preprocess_query(query_text)
    
    if len(query_terms) <= 2:
        old_query_terms = query_terms
        query_terms = preprocessor.preprocess_query(llmExpander.expand(query_text))
        #print(old_query_terms)
        #print(query_terms)
        #print("----")
        
    top_results = model.get_top_k_documents(query_terms, 30)
    top_docs = {}
    for r in top_results:
        top_docs[r[1]] = r[2]
    
    if(len(top_results) < 10):
        print(len(top_results))
        print(query_text)
        print(query_terms)    
        top_10_res = []
        for d in top_results:
            top_10_res.append((d[1], 1))
    else:
        vm = vector_model.create_model(top_docs, min_df=1)
        vm.fit()
        top_10_res = vm.find_similar(query_terms, 10)
        if len(top_10_res) == 0:
            print("NO")
            top_10_res = []
            for d in top_results[0:10]:
                top_10_res.append((d[1], 1))

    #print(top_10_res)
    
    query_index = query_data["id"]
    doc_ids = [int(r[0]) for r in top_10_res]
    
    #print(doc_ids)

    csv_string += f"{query_index},\"{doc_ids}\",-1\n"
    
with open("../../submissions/bm25_tfidf_submission.csv", "w") as f:
    f.write(csv_string)

Query Preprocessing and Expansion:   1%|▏         | 105/7437 [00:07<10:10, 12.01 queries/s]

NO


Query Preprocessing and Expansion:   3%|▎         | 233/7437 [00:21<07:41, 15.59 queries/s]

NO


Query Preprocessing and Expansion:   3%|▎         | 248/7437 [00:22<06:32, 18.33 queries/s]

...

Query Preprocessing and Expansion:  99%|█████████▊| 7343/7437 [11:26<00:06, 13.72 queries/s]

NO


Query Preprocessing and Expansion:  99%|█████████▉| 7359/7437 [11:27<00:03, 19.73 queries/s]

NO
NO
NO


Query Preprocessing and Expansion: 100%|██████████| 7437/7437 [11:33<00:00, 10.73 queries/s]


In [21]:
with open("../../submissions/bm25_tfidf_submission.csv", "w") as f:
    f.write(csv_string)

In [ ]:
for query_data in tqdm(test_queries_t2, desc="Query Preprocessing and Expansion", unit=" queries"):
    query_text = query_data["text"]
    query_terms = preprocessor.preprocess_query(query_text)